<a href="https://colab.research.google.com/github/eshnil2000/google-colab/blob/master/linear_regression_home_prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Parameters
learning_rate = 0.5
training_epochs = 
display_step = 1
batch_size=64

In [86]:
!git clone https://github.com/eshnil2000/google-colab.git
%cd google-colab
#Install required modules
!pip install tensorflow numpy matplotlib pandas sklearn
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from tqdm import tqdm

Cloning into 'google-colab'...
remote: Enumerating objects: 117, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 264 (delta 55), reused 26 (delta 9), pack-reused 147
Receiving objects: 100% (264/264), 30.33 MiB | 26.77 MiB/s, done.
Resolving deltas: 100% (120/120), done.
/content/google-colab/google-colab/google-colab/google-colab/google-colab/google-colab/google-colab


In [0]:
import pandas as pd
df = pd.read_csv('data/housepricedata.csv')
dataset = df.values
train_X = dataset[:,0:10]
train_Y = dataset[:,10]
n_batches = int(train_X.shape[0]/batch_size)

In [0]:
min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(train_X)
train_X=X_scale

In [0]:
depth=len(np.unique(train_Y))
indices=train_Y
onehot_Y=tf.one_hot(indices, depth)
with tf.Session() as sess:
    onehot_Y=sess.run(onehot_Y)
sess.close()

In [113]:
# Run optimization and test
loss_history = []
acc_history = []
optim_history=[]

X = tf.placeholder(tf.float32,shape=(batch_size,10), name="input")
Y = tf.placeholder(tf.float32,shape=(batch_size,2), name="label")
# Set model weights
# Create weights and bias
w = tf.Variable(tf.random_normal(shape=(10,2), stddev=0.1), name="weights")
b = tf.Variable(tf.zeros(shape=(1,2)), name='bias')

# calculate scores
logits = tf.matmul(X, w)+b

# Entropy cost function and loss
entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=Y)
loss = tf.reduce_mean(entropy)

# Define optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)
#optimizer= tf.train.AdamOptimizer().minimize(loss)

gd_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(entropy)

correct_mask = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_mask, tf.float32))

iterations = len(train_Y*training_epochs)

dataset = tf.data.Dataset.from_tensor_slices((train_X, onehot_Y))
dataset = dataset.repeat(training_epochs).batch(batch_size)
iterator = dataset.make_one_shot_iterator()

data_X, data_Y = iterator.get_next()
data_Y = tf.cast(data_Y, tf.int32)

c1=[]


with tf.Session() as sess, tqdm(total = iterations) as pbar:
  sess.run(tf.global_variables_initializer())
  for epoch in range(training_epochs):
    try:
      while True:
        sess.run(gd_step, feed_dict={X: data_X.eval(), Y: data_Y.eval()})
        pbar.update(batch_size)

    except tf.errors.OutOfRangeError:
      pass
  dataset = tf.data.Dataset.from_tensor_slices((train_X, onehot_Y))
  dataset = dataset.repeat(training_epochs).batch(batch_size)
  iterator = dataset.make_one_shot_iterator()

  data_X, data_Y = iterator.get_next()
  data_Y = tf.cast(data_Y, tf.int32)
  try:
    while True:
      ans = sess.run(accuracy, feed_dict={X: data_X.eval(), Y: data_Y.eval()})
  except tf.errors.OutOfRangeError:
    pass
  print("Accuracy: {:.4}%".format(ans*100))

10944it [00:03, 3626.03it/s]

Accuracy: 54.69%
